# Defining dialects with PyRDL

The PyRDL Python DSL is the intended way to define operations.
It allows to define operations directly as Python code, without relying on generating code.
It generates the operation verifiers and accessors through the use of decorators,
and also allows to export the attribute and operations as IRDL programs.
It is also type-safe, which allows its use with typing tools such as `pyright`.

This notebook shows in details all of the different features of PyRDL.

## Defining attributes

Attributes are defined in PyRDL by subclassing either the `Data` or
`ParametrizedAttribute` class with a `irdl_attr_definition` decorator.
By convention, type class names are suffixed with `Type`, and attribute
with `Attr`.

`ParametrizedAttribute` is the preffered way to define attributes, though it is
sometimes necessary to use `Data`. The difference between the two classes is
that a `ParametrizedAttribute` attribute has a tuple of `Attribute` as
parameters, while a `Data` attribute as an arbitrary Python object as parameter.

Compared to MLIR, types are `Attribute`, and any attribute that also subclass
`TypeAttribute` is defining a type.

Here is an example of a pair type definition:

In [1]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class PairType(ParametrizedAttribute, TypeAttribute):
    name = "test.pair"

    left_type: ParameterDef[Attribute]
    right_type: ParameterDef[Attribute]

print(PairType([i32, i64]))

!test.pair<i32, i64>


### Attribute name

Each attribute definition should have a unique name that has the dialect
name as prefix. For example, `test.pair` is a valid name for a pair type
or attribute in the `test` dialect.

Names are defined by overriding the `name` class field. For example, the
`test.unit` attribute is defined as:

In [2]:
from xdsl.irdl import *
from xdsl.ir import *

@irdl_attr_definition
class UnitAttr(ParametrizedAttribute):
    """An attribute with no parameters."""
    name = "test.unit"

print(UnitAttr([]))

#test.unit


### Attribute parameters

Attribute parameters are defined differently for `Data` and `ParametrizedAttribute` definitions.

#### `ParametrizedAttribute` parameters

`ParametrizedAttribute` parameters are defined using the `ParameterDef` class.
The `ParameterDef` class takes as generic argument a PyRDL constraint, which
will check that the parameter is valid at the attribute creation. The parameters
can be accessed with the `parameter` field, or can be directly accessed using the 
field name.

Here is an example of a complex attribute with two `IntegerAttr` parameters.

In [55]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class ComplexAttr(ParametrizedAttribute):
    name = "test.complex"

    real: ParameterDef[IntegerAttr]
    imaginary: ParameterDef[IntegerAttr]

attr = ComplexAttr([IntegerAttr(4, i32), IntegerAttr(-2, i32)])
print("attribute:", attr)
print("real:", attr.real)
print("imaginary:", attr.imaginary)
print("number parameters:", len(attr.parameters))


attribute: #test.complex<4 : i32, -2 : i32>
real: 4 : i32
imaginary: -2 : i32
number parameters: 2


#### `Data` parameters

`Data` parameters are defined with the generic parameter of the `Data` class.
The parameter type should be a `python` class, and is allowed to be a generic class.
The parameter can be accessed with the `data` field.

For instance, here is the definition of a string data attribute:

In [56]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.parser import *
from xdsl.printer import *

T = TypeVar("T")

@irdl_attr_definition
class StringAttr(Data[list[T]]):
    name = "test.string"

    @classmethod
    def parse_parameter(cls, parser: AttrParser) -> str:
        parser.parse_punctuation("<")
        param = parser.parse_str_literal()
        parser.parse_punctuation(">")
        return param

    def print_parameter(self, printer: Printer) -> None:
        printer.print('<"', self.data , '">')

print("attribute:", StringAttr("hello"))
print("parameter:", StringAttr("hello").data)

attribute: #test.string<"hello">
parameter: hello


### Printing and Parsing

Parsing and printing methods are defined differently for `Data` and `ParametrizedAttribute` definitions.

#### `ParametrizedAttribute` attributes

`ParametrizedAttribute` have defaults parser/printer methods that is parsing
all parameters one by one, delimited by a comma. An attribute definition can
override these methods by defining `parse_parameters` and `print_parameters`
methods:

In [35]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class ComplexAttr(ParametrizedAttribute):
    name = "test.complex"

    real: ParameterDef[IntegerAttr]
    imaginary: ParameterDef[IntegerAttr]

    @classmethod
    def parse_parameters(cls, parser: AttrParser) -> list[Attribute]:
        parser.parse_punctuation("<")
        real = parser.parse_attribute()
        parser.parse_punctuation(",")
        imaginary = parser.parse_attribute()
        parser.parse_punctuation(">")
        return [real, imaginary]

    def print_parameters(self, printer: Printer) -> None:
        printer.print("<")
        printer.print_attribute(self.real)
        printer.print(", ")
        printer.print_attribute(self.imaginary)
        printer.print(">")

ctx = MLContext()
ctx.load_dialect(Builtin)
ctx.load_attr(ComplexAttr)

attr = Parser(ctx, "#test.complex<4 : i32, -2 : i32>").parse_attribute()
print(attr)

#test.complex<4 : i32, -2 : i32>


#### `Data` attributes

`Data` attributes are required to define printing and parsing methods,
as they rely on arbitrary Python objects. A `Data` definition uses the
`parse_parameter` and `print_parameter` methods to print and parse
its parameter:

In [36]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.dialects.builtin import *

@irdl_attr_definition
class StringAttr(Data[str]):
    name = "test.string"

    @classmethod
    def parse_parameter(cls, parser: AttrParser) -> str:
        parser.parse_punctuation("<")
        param = parser.parse_str_literal()
        parser.parse_punctuation(">")
        return param

    def print_parameter(self, printer: Printer) -> None:
        printer.print('<"', self.data , '">')

ctx = MLContext()
ctx.load_dialect(Builtin)
ctx.load_attr(StringAttr)

attr = Parser(ctx, '#test.string<"foobar">').parse_attribute()
print(attr)

#test.string<"foobar">


### Additional verifiers

`ParametrizedAttribute` and `Data` can optionally define a verifier that will
be called at the attribute creation time. It is defined with the `_verify`
method.

For `ParametrizedAttribute`s, this method is called after the PyRDL constraints
that each parameter has.

In [57]:
from xdsl.irdl import *
from xdsl.ir import *
from xdsl.parser import *
from xdsl.printer import *

T = TypeVar("T")

@irdl_attr_definition
class EvenStringAttr(Data[list[T]]):
    """A string of even length"""
    name = "test.even_string"

    @classmethod
    def parse_parameter(cls, parser: AttrParser) -> str:
        parser.parse_punctuation("<")
        param = parser.parse_str_literal()
        parser.parse_punctuation(">")
        return param

    def print_parameter(self, printer: Printer) -> None:
        printer.print('<"', self.data , '">')

    def _verify(self) -> None:
        if len(self.data) % 2 != 0:
            raise VerifyException("String length must be even")

print(EvenStringAttr("even"))
try:
    EvenStringAttr("odd")
except VerifyException as e:
    print(e)

#test.even_string<"even">
String length must be even


### Inheritance

### Using generics

## Defining operations

### 

## Defining dialects

## PyRDL constraints

It allows to define operations directly as Python code, without relying on generating code. It is also type-safe, which allows its use with typing tools such as `pyright`.

This notebook shows in details all of the different features of PyRDL, but can also be used as a tutorial.

## Defining attributes

Data / ParametrizedAttribute

### Attribute name

name = ...

### Attribute parameters

#### Data parameters

#### Parametrized attr parameters

## Defining operations

### 

## Defining dialects

## PyRDL constraints